In [1]:
'''
File name                   :  Multi Augmentation
--  | Author                :  Infosense Services
--  | Creation Date         :  25-Nov-2022
  
--  DATE                        BY                   Version                DESCRIPTION
-- ------------------------------------------------------------------------------------------
--  25-Nov-2022              Vipul Prajapati           1.0              Initial Version Created 
--  08-Aug-2023              Vipul Prajapati           1.1              Optimize Code Use Py DF Instead DB Data
'''

'\nFile name                   :  Multi Augmentation\n--  | Author                :  Infosense Services\n--  | Creation Date         :  25-Nov-2022\n  \n--  DATE                        BY                   Version                DESCRIPTION\n-- ------------------------------------------------------------------------------------------\n--  25-Nov-2022              Vipul Prajapati           1.0              Initial Version Created \n--  08-Aug-2023              Vipul Prajapati           1.1              Optimize Code Use Py DF Instead DB Data\n'

In [1]:
# Library Imports

import os
import gc
import ast
import json
import time
import uuid
import math
import traceback
import warnings
import urllib.parse
import pandas as pd
import numpy as np
import psycopg2
from sqlalchemy import create_engine
import psycopg2.extras as extras
warnings.filterwarnings("ignore")

In [3]:
# DATABASE PARAMETER

database = 'isgpostgres'
user = 'postgres'
password = '123@NjMpTs'
host = '20.62.75.39'
port = '9030'

connetion = psycopg2.connect(database = database, user = user ,password = password, host = host,port = port)

# GLOBAL VARIABLE PARAMETER
volume_table = 'tm_new_data.nys_hour_8_volume_240524'
connected_ids_table = 'tm_new_data.ny_connected_osm_master_231211'
output_table = 'nys_augmentation_hour_8_volume_231211'

iteration = 1
source = 'MFA'

Missing_IDS = []
error_dict = {}

if source == 'MFA':tag ='next'
else:tag = 'prev' 

# augmented_df = pd.DataFrame({'isg_osm_id':[],'direction':[],'source':[],'highway_id':[],'volume':[],
#                              'original_ids':[],'processed_ids':[],'single_filled':[],'multi_filled':[],
#                              'iteration':[]},dtype='int64')

# selected_lst = ['isg_osm_id','direction','source','highway_id','volume',
#                'original_ids','processed_ids','single_filled','multi_filled','iteration']

augmented_df = pd.DataFrame({'isg_osm_id':[],'source':[],'highway_id':[],'volume':[],
                             'original_ids':[],'single_filled':[],'iteration':[]},dtype='int64')


selected_lst = ['isg_osm_id','source','highway_id','volume',
               'original_ids','single_filled','iteration']

# READ All BASE DATA

sql_command = f""" select distinct isg_osm_id,highway_id,region,volume 
                  from {volume_table}""" 

all_osm_data_df = pd.read_sql(sql_command, connetion)
ALL_Available_LST = np.unique(list(set(all_osm_data_df[all_osm_data_df['volume'] >= 0 ].to_dict(orient='list')['isg_osm_id']))).tolist()


# GET BASE DATA FOR AVAILABLE VOLUME

sql_command = f"""select distinct isg_osm_id,direction,highway_id,volume,source,
                         multi_{tag}_ids as connected_ids from
                    (select A.*,B.multi_{tag}_ids,B.single_{tag}_ids from 
                        (select * from {volume_table} where volume >= 3 ) A
                        ,{connected_ids_table} B
                    where A.isg_osm_id = B.current_id
                    )foo where multi_{tag}_ids !='{str('{}')}' and single_{tag}_ids ='{str('{}')}'""" 



All_Data_df = pd.read_sql(sql_command, connetion)
Updated_Available_LST = np.unique(list(set(All_Data_df.to_dict(orient='list')['isg_osm_id']))).tolist()


print("Available Volume :",len(Updated_Available_LST))
print(" ")

print("ALL Available Volume :",len(ALL_Available_LST))
print(" ")


print("Total Data Length :",len(all_osm_data_df))
print("Number Of Columns :",all_osm_data_df.columns.tolist())
print(" ")

print("Total Available Data Length :",len(All_Data_df))
print("Number Of Columns :",All_Data_df.columns.tolist())
print(" ")
print(sql_command)

Available Volume : 2228
 
ALL Available Volume : 182213
 
Total Data Length : 2321687
Number Of Columns : ['isg_osm_id', 'direction', 'highway_id', 'region', 'volume']
 
Total Available Data Length : 2228
Number Of Columns : ['isg_osm_id', 'direction', 'highway_id', 'volume', 'source', 'connected_ids']
 
select distinct isg_osm_id,direction,highway_id,volume,source,
                         multi_prev_ids as connected_ids from
                    (select A.*,B.multi_prev_ids,B.single_prev_ids from 
                        (select * from tm_new_data.nys_hour_8_volume_231211 where volume >= 3 ) A
                        ,tm_new_data.ny_connected_osm_master_231211 B
                    where A.isg_osm_id = B.current_id
                    )foo where multi_prev_ids !='{}' and single_prev_ids ='{}'


In [4]:
start = time.time()
counter = 0
for search_osm_id in Updated_Available_LST: 

    try:
        original_id = search_osm_id
        # Extract Current Search IDS Data
        extract_df = All_Data_df.loc[All_Data_df['isg_osm_id'] == search_osm_id].reset_index(drop = True)
        #print("extract_df :",extract_df)
        multi_level_ids_lst = extract_df['connected_ids'].iloc[0]
        #print("multi_level_ids_lst :",multi_level_ids_lst,len(multi_level_ids_lst))
            
        if len(multi_level_ids_lst) > 1 :

            available_lst=list(set(ALL_Available_LST).intersection(multi_level_ids_lst))
            filled_ids = list(set(multi_level_ids_lst) - set(available_lst))
            #print("available_lst :",available_lst,len(available_lst),filled_ids)
            
            if len(filled_ids) == 1:
                # Get Available Data
                Available_df = all_osm_data_df[all_osm_data_df['isg_osm_id'].isin(available_lst)]
                # Filled Data = Current Data - Available Data
                # check for current volume should be greater than available volume
                if extract_df['volume'].iloc[0] > Available_df['volume'].sum().item():
                
                    extract_df['volume'] = abs(extract_df['volume'] - Available_df['volume'].sum())
                    # Check If Current Volume After Subtracted From Available Volume Is 0 Or Not
                    extract_df = extract_df.head(1)
                    multi_level_df = extract_df
                    multi_level_df.columns = extract_df.columns
                    multi_level_df = multi_level_df.dropna()
                    multi_level_df['isg_osm_id'] = filled_ids
                    multi_level_df['source'] = source
                    multi_level_df['original_ids'] = original_id
                    #multi_level_df['processed_ids'] = search_osm_id
                    multi_level_df['single_filled'] = None
                    #multi_level_df['multi_filled'] = str(filled_ids)
                    multi_level_df['iteration'] = iteration  

                    multi_level_df = multi_level_df[selected_lst]
                    #print("multi_level_df :",multi_level_df)
                    augmented_df = pd.concat([augmented_df, multi_level_df], ignore_index=True)

        counter = counter + 1
        print("Processed ID :",counter)
        
    except:
        #print(traceback.format_exc())
        error_dict[str(search_osm_id)] = str(traceback.format_exc())
        Missing_IDS.append(search_osm_id)
        print("Some ID Missed")
        continue
        
end = time.time()

Processed ID : 1
Processed ID : 2
Processed ID : 3
Processed ID : 4
Processed ID : 5
Processed ID : 6
Processed ID : 7
Processed ID : 8
Processed ID : 9
Processed ID : 10
Processed ID : 11
Processed ID : 12
Processed ID : 13
Processed ID : 14
Processed ID : 15
Processed ID : 16
Processed ID : 17
Processed ID : 18
Processed ID : 19
Processed ID : 20
Processed ID : 21
Processed ID : 22
Processed ID : 23
Processed ID : 24
Processed ID : 25
Processed ID : 26
Processed ID : 27
Processed ID : 28
Processed ID : 29
Processed ID : 30
Processed ID : 31
Processed ID : 32
Processed ID : 33
Processed ID : 34
Processed ID : 35
Processed ID : 36
Processed ID : 37
Processed ID : 38
Processed ID : 39
Processed ID : 40
Processed ID : 41
Processed ID : 42
Processed ID : 43
Processed ID : 44
Processed ID : 45
Processed ID : 46
Processed ID : 47
Processed ID : 48
Processed ID : 49
Processed ID : 50
Processed ID : 51
Processed ID : 52
Processed ID : 53
Processed ID : 54
Processed ID : 55
Processed ID : 56
P

Processed ID : 448
Processed ID : 449
Processed ID : 450
Processed ID : 451
Processed ID : 452
Processed ID : 453
Processed ID : 454
Processed ID : 455
Processed ID : 456
Processed ID : 457
Processed ID : 458
Processed ID : 459
Processed ID : 460
Processed ID : 461
Processed ID : 462
Processed ID : 463
Processed ID : 464
Processed ID : 465
Processed ID : 466
Processed ID : 467
Processed ID : 468
Processed ID : 469
Processed ID : 470
Processed ID : 471
Processed ID : 472
Processed ID : 473
Processed ID : 474
Processed ID : 475
Processed ID : 476
Processed ID : 477
Processed ID : 478
Processed ID : 479
Processed ID : 480
Processed ID : 481
Processed ID : 482
Processed ID : 483
Processed ID : 484
Processed ID : 485
Processed ID : 486
Processed ID : 487
Processed ID : 488
Processed ID : 489
Processed ID : 490
Processed ID : 491
Processed ID : 492
Processed ID : 493
Processed ID : 494
Processed ID : 495
Processed ID : 496
Processed ID : 497
Processed ID : 498
Processed ID : 499
Processed ID

Processed ID : 891
Processed ID : 892
Processed ID : 893
Processed ID : 894
Processed ID : 895
Processed ID : 896
Processed ID : 897
Processed ID : 898
Processed ID : 899
Processed ID : 900
Processed ID : 901
Processed ID : 902
Processed ID : 903
Processed ID : 904
Processed ID : 905
Processed ID : 906
Processed ID : 907
Processed ID : 908
Processed ID : 909
Processed ID : 910
Processed ID : 911
Processed ID : 912
Processed ID : 913
Processed ID : 914
Processed ID : 915
Processed ID : 916
Processed ID : 917
Processed ID : 918
Processed ID : 919
Processed ID : 920
Processed ID : 921
Processed ID : 922
Processed ID : 923
Processed ID : 924
Processed ID : 925
Processed ID : 926
Processed ID : 927
Processed ID : 928
Processed ID : 929
Processed ID : 930
Processed ID : 931
Processed ID : 932
Processed ID : 933
Processed ID : 934
Processed ID : 935
Processed ID : 936
Processed ID : 937
Processed ID : 938
Processed ID : 939
Processed ID : 940
Processed ID : 941
Processed ID : 942
Processed ID

Processed ID : 1314
Processed ID : 1315
Processed ID : 1316
Processed ID : 1317
Processed ID : 1318
Processed ID : 1319
Processed ID : 1320
Processed ID : 1321
Processed ID : 1322
Processed ID : 1323
Processed ID : 1324
Processed ID : 1325
Processed ID : 1326
Processed ID : 1327
Processed ID : 1328
Processed ID : 1329
Processed ID : 1330
Processed ID : 1331
Processed ID : 1332
Processed ID : 1333
Processed ID : 1334
Processed ID : 1335
Processed ID : 1336
Processed ID : 1337
Processed ID : 1338
Processed ID : 1339
Processed ID : 1340
Processed ID : 1341
Processed ID : 1342
Processed ID : 1343
Processed ID : 1344
Processed ID : 1345
Processed ID : 1346
Processed ID : 1347
Processed ID : 1348
Processed ID : 1349
Processed ID : 1350
Processed ID : 1351
Processed ID : 1352
Processed ID : 1353
Processed ID : 1354
Processed ID : 1355
Processed ID : 1356
Processed ID : 1357
Processed ID : 1358
Processed ID : 1359
Processed ID : 1360
Processed ID : 1361
Processed ID : 1362
Processed ID : 1363


Processed ID : 1736
Processed ID : 1737
Processed ID : 1738
Processed ID : 1739
Processed ID : 1740
Processed ID : 1741
Processed ID : 1742
Processed ID : 1743
Processed ID : 1744
Processed ID : 1745
Processed ID : 1746
Processed ID : 1747
Processed ID : 1748
Processed ID : 1749
Processed ID : 1750
Processed ID : 1751
Processed ID : 1752
Processed ID : 1753
Processed ID : 1754
Processed ID : 1755
Processed ID : 1756
Processed ID : 1757
Processed ID : 1758
Processed ID : 1759
Processed ID : 1760
Processed ID : 1761
Processed ID : 1762
Processed ID : 1763
Processed ID : 1764
Processed ID : 1765
Processed ID : 1766
Processed ID : 1767
Processed ID : 1768
Processed ID : 1769
Processed ID : 1770
Processed ID : 1771
Processed ID : 1772
Processed ID : 1773
Processed ID : 1774
Processed ID : 1775
Processed ID : 1776
Processed ID : 1777
Processed ID : 1778
Processed ID : 1779
Processed ID : 1780
Processed ID : 1781
Processed ID : 1782
Processed ID : 1783
Processed ID : 1784
Processed ID : 1785


Processed ID : 2162
Processed ID : 2163
Processed ID : 2164
Processed ID : 2165
Processed ID : 2166
Processed ID : 2167
Processed ID : 2168
Processed ID : 2169
Processed ID : 2170
Processed ID : 2171
Processed ID : 2172
Processed ID : 2173
Processed ID : 2174
Processed ID : 2175
Processed ID : 2176
Processed ID : 2177
Processed ID : 2178
Processed ID : 2179
Processed ID : 2180
Processed ID : 2181
Processed ID : 2182
Processed ID : 2183
Processed ID : 2184
Processed ID : 2185
Processed ID : 2186
Processed ID : 2187
Processed ID : 2188
Processed ID : 2189
Processed ID : 2190
Processed ID : 2191
Processed ID : 2192
Processed ID : 2193
Processed ID : 2194
Processed ID : 2195
Processed ID : 2196
Processed ID : 2197
Processed ID : 2198
Processed ID : 2199
Processed ID : 2200
Processed ID : 2201
Processed ID : 2202
Processed ID : 2203
Processed ID : 2204
Processed ID : 2205
Processed ID : 2206
Processed ID : 2207
Processed ID : 2208
Processed ID : 2209
Processed ID : 2210
Processed ID : 2211


In [5]:
print("TIME TAKEN :",(end - start))

TIME TAKEN : 23.184823989868164


In [6]:
print(len(augmented_df['isg_osm_id'].unique()))
print(len(augmented_df['isg_osm_id']))
print(len(Missing_IDS))
print(len(error_dict.keys()))

708
708
0
0


In [7]:
augmented_df.head()

,isg_osm_id,direction,source,highway_id,volume,original_ids,single_filled,iteration
0,24137110001,E,MBA,2,859,4350451001,None,1
1,4371845001,N,MBA,2,1018,4371845002,None,1
2,5566842005,N,MBA,5,147,5585376001,None,1
3,196948817002,E,MBA,2,241,5619033001,None,1
4,377881019001,S,MBA,3,211,5628863001,None,1


In [8]:
# Insert Data Into DB

database = 'isgpostgres'
user = 'postgres'
password = '123@NjMpTs'
host = '20.62.75.39'
port = '9030'

encoded_password = urllib.parse.quote(password)

schema_name = 'tm_new_data'
table_name = output_table

connection_string = "postgresql://" + user + ":" + encoded_password + "@" + host + ":" + port + "/" + database
engine = create_engine(connection_string)

augmented_df.to_sql(table_name,engine,if_exists='append',schema=schema_name, chunksize=10000,index=False,method='multi')

708

In [9]:
# del [[all_osm_data_df,All_Data_df,Updated_Available_LST,ALL_Available_LST,augmented_df]]
# gc.collect()

126